In [1]:
# import libraries

import os
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import mygene
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score, precision_recall_curve
import random
from random import randint
from sklearn.model_selection import StratifiedKFold
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# define directories

cell_line_dir = "/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC/"
models_dir = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/PDX_DNA_REPi/Expression_CNA/Models/"
DEGs_dir = "/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/DNA_REPi/"
dataset_dir = "/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/"
pdx_data_dir = "/Volumes/Expansion/Thesis Work/Datasets/PDX/Expression/"
save_results_to = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/PDX_DNA_REPi/Expression_CNA/Predictions/"

In [3]:
# set random seeds

torch.manual_seed(42)
random.seed(42)

In [4]:
# change directory to read GDSC cell line details

os.chdir(cell_line_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC


In [5]:
# read cell line details table in GDSC

GDSC_cell_line_details = pd.read_excel("GDSC_Cell_Lines_Details.xlsx", keep_default_na = False)
GDSC_cell_line_details.set_index("COSMIC identifier", inplace = True)
GDSC_cell_line_details.tail()

,Sample Name,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
COSMIC identifier,,,,,,,,,,,,
1298218,KP-2,Y,N,Y,N,Y,pancreas,pancreas,PAAD,,D/F12,Adherent
1330932,KO52,Y,Y,N,N,N,leukemia,acute_myeloid_leukaemia,LAML,MSS/MSI-L,D/F12,NA
1331030,SC-1,Y,Y,N,N,N,lymphoma,B_cell_lymphoma,DLBC,MSS/MSI-L,R,NA
1503373,U-CH2,Y,Y,N,N,N,bone,bone_other,,MSS/MSI-L,D/F12,Adherent
,TOTAL:,1001,996,968,957,990,,,,,,


In [6]:
# change directory to read GDSC expression dataset (DNA_REPi)

os.chdir(dataset_dir + "/exprs/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs


In [7]:
# read GDSC expression dataset (DNA_REPi)

GDSCE = pd.read_csv("GDSC_exprs.z.DNA_REPi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
GDSCE = pd.DataFrame.transpose(GDSCE)
GDSCE.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,107984199,107984208,107984325,107984411,107984648,107984889,107984989,107986313,107986782,107986810
683665,-0.4711563767951986,-0.1857522957766964,0.9702409548662462,-0.4081281805960821,-0.4161511169999803,-0.4384159723467665,-1.154692246005183,0.5262132484107975,-0.7950924576033422,-0.6337442716864214,...,1.283534690590172,-0.3367265464586691,-0.0039762988723447,0.7378729872604209,0.9757614264575688,0.1494417647095801,2.062294063911948,-0.8140636761719959,0.2870333828833719,0.9524265085537148
684055,1.3565140948247052,-0.277542411913307,0.0887087882196536,-0.2754829982302413,-0.5153856346587746,-0.6248712076433911,0.1243408990866293,-1.2699667684674136,2.0788360886382717,2.5197991997459197,...,-0.5571028630154293,-1.724090733574364,-0.3749080365068901,-0.0016055431935905,-0.3788535663547365,-1.3502092798315848,0.6949790577550328,0.4333032198982747,-0.1280986270308099,-1.629249958712223
684057,0.6510004960254864,1.6578876382433665,-0.4977106390881842,-0.1212172092933644,-0.398455281697677,-0.2804901106672752,0.6455898116420014,0.215833091774836,-0.1723595788875009,-0.2170870770552223,...,-0.024863133395904,0.4073439017665206,-0.4440356638831038,0.8630347078663075,1.4486171428489227,-1.4777214830771732,0.5220400480464272,0.6441211958947753,0.9057518606131394,0.3018783753479829


In [8]:
# change directory to read GDSC CNA dataset (DNA_REPi)

os.chdir(dataset_dir + "/CNA/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/CNA


In [9]:
# read GDSC CNA dataset (DNA_REPi)

GDSCC = pd.read_csv("GDSC_CNA.DNA_REPi.tsv",
                    sep="\t", index_col=0, decimal=".")
GDSCC.drop_duplicates(keep='last')
GDSCC = pd.DataFrame.transpose(GDSCC)
GDSCC = GDSCC.loc[:, ~GDSCC.columns.duplicated()]

GDSCC.head()

gene_id,1,2,9,10,12,13,14,15,16,18,...,107985535,107985759,107986588,107986809,107986898,107987337,107987341,109731405,112441434,121676927
683665,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,1.807355,0.000000,0.000000,0.000000
684055,0.000000,0.584963,0.584963,0.584963,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.584963,0.000000,0.584963,0.584963,-4.32,-4.320000,0.000000,0.000000,0.584963
684057,-0.415037,0.000000,0.584963,0.584963,-1.000000,0.000000,0.0,0.0,-0.415037,-0.415037,...,-0.415037,0.000000,0.000000,0.000000,0.807355,-1.00,-1.000000,-0.415037,-0.415037,0.807355
684059,0.000000,0.000000,0.000000,0.000000,0.000000,0.415037,0.0,0.0,0.000000,0.415037,...,0.000000,0.000000,0.415037,0.000000,0.415037,-4.32,-4.320000,0.000000,0.000000,0.736966
684062,-0.415037,0.000000,0.321928,0.321928,-0.415037,-0.415037,0.0,0.0,-0.415037,0.000000,...,-0.415037,0.000000,0.000000,0.000000,0.321928,-4.32,-4.320000,-0.415037,-1.000000,0.321928


In [10]:
# change directory to read GDSC response dataset (DNA_REPi)

os.chdir(dataset_dir + "/response/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response


In [11]:
# read GDSC response dataset (DNA_REPi)

GDSCR = pd.read_csv("GDSC_response.DNA_REPi.tsv",
                    sep="\t",
                    index_col=0,
                    decimal=",")
GDSCR.dropna(inplace=True)
GDSCR.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
GDSCR["response"] = GDSCR.loc[:, "response"].apply(lambda x: d[x])

GDSCR.head()

,response,logIC50,drug,exprs,CNA,mutations
sample_name,,,,,,
683665,0,5.12075503641523,Pyrimethamine,1,1,1
683665,0,-3.03711635474651,Doxorubicin,1,1,1
683665,0,-0.712119156515401,Etoposide,1,1,1
683665,0,-4.40897202584936,Gemcitabine,1,1,1
683665,0,-1.59752443691324,Mitomycin C,1,1,1


In [12]:
# print count of cell lines for each drug (DNA_REPi)

GDSCR["drug"].value_counts()

drug
Etoposide         775
SN-38             771
Mitomycin C       767
Doxorubicin       766
5-Fluorouracil    764
Gemcitabine       761
Bleomycin         759
Temozolomide      752
Cisplatin         738
Methotrexate      738
Camptothecin      737
Cytarabine        735
Pyrimethamine     372
Name: count, dtype: int64

In [13]:
# change directory to read PDX expression dataset homogenized with GDSC expression dataset (5-Fluorouracil)

os.chdir(dataset_dir + "/exprs_homogenized/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs_homogenized


In [14]:
# read PDX expression dataset homogenized with GDSC expression dataset (5-Fluorouracil)

PDXEfluorouracil = pd.read_csv("PDX_exprs.5-Fluorouracil.eb_with.GDSC_exprs.5-Fluorouracil.tsv",
                   sep = "\t", index_col=0, decimal = ",")
PDXEfluorouracil = pd.DataFrame.transpose(PDXEfluorouracil)
PDXEfluorouracil.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,100507206,100507254,100507436,100507472,100526773,100527978,100532746,100820829,102724473,105375355
X-1027,4.11649564280422,3.59209496158439,6.28559101273304,2.67926549063284,3.04621480878604,2.69044790660834,8.01420464403606,2.73140674359613,8.71768731731273,2.89488898093329,...,2.71033163066003,3.2543218456306,5.67560198356418,3.07143466694584,3.0597754133993,2.63635721806084,2.61292331163008,3.18623983789074,0,2.4522722382887
X-1119,4.00318976265845,3.59209496158439,7.55375516379688,4.03669966422185,3.41557516120483,5.94712898968845,8.14610109582064,2.54373640699005,7.8874359290969,3.43325748497387,...,2.80929358976047,3.3020335598854,8.94930072777643,3.07143466694584,3.0597754133993,2.63635721806084,2.61292331163008,4.60199637750889,0,2.89512503666773
X-1173,3.74198340066366,3.67493736139833,6.16750446793135,3.50545765081401,3.04295501121306,2.98779164904873,8.37697489020915,2.64942372799293,8.2967442081863,4.13640544224621,...,2.71033163066003,3.2543218456306,4.67270724878415,3.07143466694584,3.0597754133993,2.63635721806084,2.61292331163008,4.99706823903714,0,3.34491841640924


In [15]:
# change directory to read PDX CNA dataset (5-Fluorouracil)

os.chdir(dataset_dir + "CNA")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/CNA'

In [16]:
# read PDX CNA dataset (5-Fluorouracil)

PDXCfluorouracil = pd.read_csv("PDX_CNA.5-Fluorouracil.tsv",
                        sep="\t", index_col=0, decimal=",")
PDXCfluorouracil.drop_duplicates(keep='last')
PDXCfluorouracil = pd.DataFrame.transpose(PDXCfluorouracil)
PDXCfluorouracil = PDXCfluorouracil.loc[:, ~PDXCfluorouracil.columns.duplicated()]

PDXCfluorouracil.head(3)

ENTREZID,1,2,3,9,10,12,13,14,15,16,...,101340252,102723547,102724473,103091865,105375355,107126285,109623460,109731405,118126072,128385369
X-1027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.1844245711374275,0.0,0.0,0.0,0.0
X-1119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.7311832415721999,0.0,0.0,0.0,0.4276061727818994,0.0
X-1173,0.0,0.0,0.0,0.0,0.0,0.3391373849195853,0.0,0.3504972470841331,0.0,0.0,...,0.3391373849195853,0.3785116232537298,0.3673710656485294,0.0,0.8196681834964555,0.3617683594191533,0.0,0.0,0.3219280948873623,0.0


In [17]:
# variance threshold for GDSC Expression dataset (DNA_REPi)

selector = VarianceThreshold(0.05)
selector.fit_transform(GDSCE)
GDSCE = GDSCE[GDSCE.columns[selector.get_support(indices=True)]]

In [18]:
# fill NA values and binarize GDSC CNA dataset (DNA_REPi)

GDSCC = GDSCC.fillna(0)
GDSCC[GDSCC != 0.0] = 1

In [19]:
# select shared genes between GDSC and PDX expression and CNA datasets 

ls = GDSCE.columns.intersection(GDSCC.columns)
ls = ls.intersection(PDXEfluorouracil.columns)
ls = ls.intersection(PDXCfluorouracil.columns)

ls = pd.unique(ls)

print(f"GDSC and PDX # of common genes: {ls.shape[0]}")

GDSC and PDX # of common genes: 17901


In [20]:
# select shared samples between GDSC expression and response datasets (DNA_REPi)

ls2 = GDSCE.index.intersection(GDSCC.index)
ls2 = ls2.intersection(GDSCR.index)

print(f"GDSC # of common cell lines: {ls2.shape[0]}")

GDSC # of common cell lines: 801


In [21]:
# subset shared genes and samples in GDSC expression, CNA, and response datasets

GDSCE = GDSCE.loc[ls2, ls]
GDSCC = GDSCC.loc[ls2, ls]
GDSCR = GDSCR.loc[ls2, :]

print(f"GDSC # of common samples and genes (Expression): {GDSCE.shape}")
print(f"GDSC # of common samples and genes (CNA): {GDSCC.shape}")
print(f"GDSC # of samples (Response): {GDSCR.shape[0]}")

# There exists same sample names for different drugs, so row shape is different in GDSC response data

GDSC # of common samples and genes (Expression): (801, 17901)
GDSC # of common samples and genes (CNA): (801, 17901)
GDSC # of samples (Response): 9435


In [22]:
# select shared samples between PDX expression and CNA datasets (5-Fluorouracil) 

ls3 = PDXEfluorouracil.index.intersection(PDXCfluorouracil.index)

print(f"PDX # of common samples (5-Fluorouracil): {ls3.shape[0]}")

PDX # of common samples (5-Fluorouracil): 23


In [23]:
# select shared genes and samples between PDX expression and CNA datasets (5-Fluorouracil)

PDXEfluorouracil = PDXEfluorouracil.loc[ls3, ls]
PDXCfluorouracil = PDXCfluorouracil.loc[ls3, ls]

print(f"PDX # of common samples and genes for Expression (5-Fluorouracil): {PDXEfluorouracil.shape}")
print(f"PDX # of common samples and genes for CNA (5-Fluorouracil): {PDXCfluorouracil.shape}\n")

PDX # of common samples and genes for Expression (5-Fluorouracil): (23, 17901)
PDX # of common samples and genes for CNA (5-Fluorouracil): (23, 17901)



In [24]:
# change directory to read DEGs (DNA_REPi)

os.chdir(DEGs_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/DNA_REPi


In [25]:
# read DEGs (DNA_REPi)

DEGs_filtered_data = pd.read_excel("DNA_REPi_Differentially_Expressed_Genes (EnsemblID).xlsx",
                                    sheet_name = "Common DEGs")
print(f"There are {DEGs_filtered_data.shape[0]} DEGs (p-adjusted < 0.05 and |logFC| > 1) among DNA replication inhibitors\n")

DEGs_filtered_data.head()

There are 858 DEGs (p-adjusted < 0.05 and |logFC| > 1) among DNA replication inhibitors



,Gene Symbol,Frequency
0,SLFN11,9
1,NCKAP1L,8
2,SDC4,7
3,KRT18,7
4,S100A14,7


In [26]:
# get gene symbol from mygene database

mg = mygene.MyGeneInfo()
DEGs_entrez_id = mg.querymany(DEGs_filtered_data["Gene Symbol"], 
                              species="human", 
                              scopes = "symbol",
                              field = "entrezgene",
                              as_dataframe=True)["entrezgene"]
DEGs_entrez_id = pd.unique(DEGs_entrez_id.dropna())
DEGs_entrez_id.shape[0]

1 input query terms found dup hits:	[('CLECL1P', 2)]


858

In [27]:
# assign GDSC datasets to new variables

exprs_z = GDSCE
cna = GDSCC
responses = GDSCR

In [28]:
# GDSC response dataset

responses.head(3)

,response,logIC50,drug,exprs,CNA,mutations
683665,0,5.12075503641523,Pyrimethamine,1,1,1
683665,0,-3.03711635474651,Doxorubicin,1,1,1
683665,0,-0.712119156515401,Etoposide,1,1,1


In [29]:
# list drug names in GDSC response dataset

drugs = set(responses["drug"].values)
drugs

{'5-Fluorouracil',
 'Bleomycin',
 'Camptothecin',
 'Cisplatin',
 'Cytarabine',
 'Doxorubicin',
 'Etoposide',
 'Gemcitabine',
 'Methotrexate',
 'Mitomycin C',
 'Pyrimethamine',
 'SN-38',
 'Temozolomide'}

In [30]:
# convert Entrez gene IDs to integer

responses.index = responses.index.astype(int)
responses.index

Index([ 683665,  683665,  683665,  683665,  683665,  683665,  683665,  683665,
        683665,  683665,
       ...
       1659818, 1659818, 1659818, 1659818, 1659819, 1659819, 1659819, 1659819,
       1659819,  687800],
      dtype='int64', length=9435)

In [31]:
# merge GDSC cell line details with response dataset

dat = pd.merge(GDSC_cell_line_details,
               responses,
               left_index = True, 
               right_index=True,
               how = "inner")

dat["GDSC\nTissue descriptor 1"].value_counts()
dat.head()

,Sample Name,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties,response,logIC50,drug,exprs,CNA,mutations
COSMIC identifier,,,,,,,,,,,,,,,,,,
906794,A253,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,,MSS/MSI-L,D/F12,Adherent,0,3.25484543425934,Pyrimethamine,1,1,1
906794,A253,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,,MSS/MSI-L,D/F12,Adherent,0,-3.08563930086988,Doxorubicin,1,1,1
906794,A253,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,,MSS/MSI-L,D/F12,Adherent,1,-2.64209579180854,Etoposide,1,1,1
906794,A253,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,,MSS/MSI-L,D/F12,Adherent,0,-2.92273746018911,Gemcitabine,1,1,1
906794,A253,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,,MSS/MSI-L,D/F12,Adherent,0,-1.0950578547037,Mitomycin C,1,1,1


In [32]:
# list count of cell lines as to tissues treated with 5-Fluorouracil

filter = (dat["drug"] == "5-Fluorouracil")
dat.loc[filter,"GDSC\nTissue descriptor 1"].value_counts()

GDSC\nTissue descriptor 1
lung_NSCLC           86
urogenital_system    78
leukemia             63
aero_dig_tract       59
lymphoma             57
nervous_system       50
skin                 47
lung_SCLC            46
breast               44
large_intestine      38
digestive_system     37
bone                 34
neuroblastoma        28
pancreas             25
kidney               18
lung                 15
soft_tissue          14
myeloma              13
thyroid              12
Name: count, dtype: int64

In [33]:
# convert Entrez gene IDs to string

responses.index = responses.index.astype(str)
responses.index

Index(['683665', '683665', '683665', '683665', '683665', '683665', '683665',
       '683665', '683665', '683665',
       ...
       '1659818', '1659818', '1659818', '1659818', '1659819', '1659819',
       '1659819', '1659819', '1659819', '687800'],
      dtype='object', length=9435)

In [34]:
# filter GDSC expression and CNA datasets (DNA_REPi) as to drugs
# subset selected DEGs

expression_zscores = []
CNA = []
for drug in drugs:
    samples = responses.loc[responses["drug"] == drug, :].index.values
    e_z = exprs_z.loc[samples, :]
    c = cna.loc[samples, :]
    expression_zscores.append(e_z)
    CNA.append(c)

GDSCEv2 = pd.concat(expression_zscores, axis=0)
GDSCCv2 = pd.concat(CNA, axis=0)
GDSCRv2 = responses

ls4 = list(set(GDSCE.columns).intersection(set(DEGs_entrez_id.astype(int))))
ls5 = GDSCEv2.index.intersection(GDSCCv2.index)

GDSCEv2 = GDSCEv2.loc[ls5, ls4]
GDSCCv2 = GDSCCv2.loc[ls5, ls4]
GDSCRv2 = GDSCRv2.loc[ls5, :]

PDXEfluorouracil = PDXEfluorouracil.loc[:,ls4]
PDXCfluorouracil = PDXCfluorouracil.loc[:,ls4]

responses.index = responses.index.values + "_" + responses["drug"].values

print(f"GDSC # of common samples and genes (Expression): {GDSCEv2.shape}")
print(f"GDSC # of common samples and genes (CNA): {GDSCCv2.shape}")
print(f"GDSC # of common samples (Response): {GDSCRv2.shape[0]}\n")

print(f"PDX # of common samples and genes for 5-Fluorouracil (Expression): {PDXEfluorouracil.shape}")
print(f"PDX # of common samples and genes for 5-Fluorouracil (CNA): {PDXCfluorouracil.shape}\n")

GDSC # of common samples and genes (Expression): (9435, 849)
GDSC # of common samples and genes (CNA): (9435, 849)
GDSC # of common samples (Response): 9435

PDX # of common samples and genes for 5-Fluorouracil (Expression): (23, 849)
PDX # of common samples and genes for 5-Fluorouracil (CNA): (23, 849)



In [35]:
# GDSC response values

Y = GDSCRv2['response'].values
print(Y)

[0 0 0 ... 0 1 0]


In [36]:
# change directory to read PDX response dataset (5-Fluorouracil)

os.chdir(dataset_dir + "response")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response'

In [37]:
# read PDX response dataset (5-Fluorouracil)

PDXRfluorouracil = pd.read_csv("PDX_response.5-Fluorouracil.tsv", 
                      sep = "\t", 
                      index_col=0, 
                      decimal = ",")
PDXRfluorouracil.dropna(inplace=True)
PDXRfluorouracil.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
PDXRfluorouracil["response"] = PDXRfluorouracil.loc[:, "response"].apply(lambda x: d[x])
Ytsfluorouracil = PDXRfluorouracil["response"].values    

print(f"There are {Ytsfluorouracil.shape[0]} samples with response data in the PDX (5-FU)\n")
Ytsfluorouracil

There are 23 samples with response data in the PDX (5-FU)



array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [38]:
# read PDX dataset to select cohorts (5-Fluorouracil)

pdx_cohort = pd.read_excel(pdx_data_dir + "/nm.3954-S2.xlsx",
                              "PCT raw data")
pdx_cohort = pdx_cohort.loc[pdx_cohort["Treatment"] == "5FU",]
pdx_cohort = pdx_cohort.loc[:,["Model","Tumor Type"]]
pdx_cohort = pdx_cohort.drop_duplicates()
pdx_cohort.set_index("Model", inplace = True) 
pdx_cohort.head()

,Tumor Type
Model,
X-0933,CRC
X-1027,CRC
X-1055,CRC
X-1119,CRC
X-1167,CRC


In [39]:
# merged PDX cohort table with PDX response dataset (5-Fluorouracil)

dat_pdx = pd.merge(pdx_cohort,
                   PDXRfluorouracil,
                   left_index = True,
                   right_index = True)
dat_pdx["Tumor Type"].value_counts()

Tumor Type
CRC    23
Name: count, dtype: int64

In [40]:
# filter CRC cohort samples (5-Fluorouracil)

filter = (dat_pdx["Tumor Type"] == "CRC")
pdx_crc_samples = dat_pdx.loc[filter,].index
pdx_crc_samples

Index(['X-1027', 'X-1119', 'X-1173', 'X-1270', 'X-1329', 'X-2145', 'X-2182',
       'X-2483', 'X-2538', 'X-2573', 'X-2659', 'X-2846', 'X-2861', 'X-3093',
       'X-3205', 'X-3267', 'X-3671', 'X-3792', 'X-4087', 'X-5405', 'X-5438',
       'X-5495', 'X-5578'],
      dtype='object')

In [41]:
# filter responses of CRC cohort samples (5-Fluorouracil)

Ytsfluorouracil_crc = PDXRfluorouracil.loc[pdx_crc_samples,"response"].values
Ytsfluorouracil_crc

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [42]:
# count of sensitive and resistance CRC samples (5-Fluorouracil)

PDXRfluorouracil.loc[pdx_crc_samples,"response"].value_counts()

response
0    22
1     1
Name: count, dtype: int64

In [43]:
# define maximum iteration

max_iter = 50

In [44]:
# assign number of neurons and dropout rates found for layers from the saved models

hdm1 = 512
hdm2 = 128
rate1 = 0.6
rate2 = 0.6
rate3 = 0.7

In [46]:
# load pre-trained models and make predictions

scalerGDSC = sk.StandardScaler()
scalerGDSC.fit(GDSCEv2.values)
X_trainE = scalerGDSC.transform(GDSCEv2.values)
X_testEfluorouracil = scalerGDSC.transform(PDXEfluorouracil.values)    
X_testEfluorouracil_crc = scalerGDSC.transform(PDXEfluorouracil.loc[pdx_crc_samples,].values) 

X_trainC = np.nan_to_num(GDSCCv2.values)
X_testCfluorouracil = np.nan_to_num(PDXCfluorouracil.values.astype(np.float32))
X_testCfluorouracil_crc = np.nan_to_num(PDXCfluorouracil.loc[pdx_crc_samples,].values.astype(np.float32))

TX_testEfluorouracil = torch.FloatTensor(X_testEfluorouracil)
TX_testEfluorouracil_crc = torch.FloatTensor(X_testEfluorouracil_crc)

TX_testCfluorouracil = torch.FloatTensor(X_testCfluorouracil.astype(np.float32))
TX_testCfluorouracil_crc = torch.FloatTensor(X_testCfluorouracil_crc.astype(np.float32))

ty_testEfluorouracil = torch.FloatTensor(Ytsfluorouracil.astype(int))
ty_testEfluorouracil_crc = torch.FloatTensor(Ytsfluorouracil_crc.astype(int))


n_sampE, IE_dim = X_trainE.shape
n_sampC, IC_dim = X_trainC.shape

h_dim1 = hdm1
h_dim2 = hdm2
Z_in = h_dim1 + h_dim2 

class AEE(nn.Module):
    def __init__(self):
        super(AEE, self).__init__()
        self.EnE = torch.nn.Sequential(
            nn.Linear(IE_dim, h_dim1),
            nn.BatchNorm1d(h_dim1),
            nn.ReLU(),
            nn.Dropout(rate1))
    def forward(self, x):
        output = self.EnE(x)
        return output  


class AEC(nn.Module):
    def __init__(self):
        super(AEC, self).__init__()
        self.EnC = torch.nn.Sequential(
            nn.Linear(IC_dim, h_dim2),
            nn.BatchNorm1d(h_dim2),
            nn.ReLU(),
            nn.Dropout(rate2))
    def forward(self, x):
        output = self.EnC(x)
        return output       

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.FC = torch.nn.Sequential(
            nn.Linear(Z_in, 1),
            nn.Dropout(rate3),
            nn.Sigmoid())
    def forward(self, x):
        return self.FC(x)

torch.cuda.manual_seed_all(42)

AutoencoderE = torch.load(models_dir + "Exprs_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA.pt")
AutoencoderC = torch.load(models_dir + "CNA_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA.pt")
Clas = torch.load(models_dir + "Class_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA.pt")

AutoencoderE.eval()
AutoencoderC.eval()
Clas.eval()

ZEX = AutoencoderE(torch.FloatTensor(X_trainE))
ZCX = AutoencoderC(torch.FloatTensor(X_trainC))
ZTX = torch.cat((ZEX, ZCX), 1)
ZTX = F.normalize(ZTX, p=2, dim=0)
PredX = Clas(ZTX)
AUCT = roc_auc_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUC (DNA_REPi): {round(AUCT,2)}")
AUCTaucpr = average_precision_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUCPR (DNA_REPi): {round(AUCTaucpr,2)}")

ZETfluorouracil = AutoencoderE(TX_testEfluorouracil)
ZCTfluorouracil = AutoencoderC(TX_testCfluorouracil)
ZTTfluorouracil = torch.cat((ZETfluorouracil, ZCTfluorouracil), 1)
ZTTfluorouracil = F.normalize(ZTTfluorouracil, p=2, dim=0)
PredTfluorouracil = Clas(ZTTfluorouracil)
AUCTfluorouracil = roc_auc_score(Ytsfluorouracil.astype(int), PredTfluorouracil.detach().numpy())
#print(f"PDX - Test AUC (5-FU): {round(AUCTfluorouracil,2)}")
AUCTfluorouracil_aucpr = average_precision_score(Ytsfluorouracil.astype(int), PredTfluorouracil.detach().numpy())
#print(f"PDX - Test AUCPR (5-FU): {round(AUCTfluorouracil_aucpr,2)}")

ZETfluorouracil_crc = AutoencoderE(TX_testEfluorouracil_crc)
ZCTfluorouracil_crc = AutoencoderC(TX_testCfluorouracil_crc)
ZTTfluorouracil_crc = torch.cat((ZETfluorouracil_crc, ZCTfluorouracil_crc), 1)
ZTTfluorouracil_crc = F.normalize(ZTTfluorouracil_crc, p=2, dim=0)
PredTfluorouracil_crc = Clas(ZTTfluorouracil_crc)
AUCTfluorouracil_crc = roc_auc_score(Ytsfluorouracil_crc.astype(int), PredTfluorouracil_crc.detach().numpy())
#print(f"PDX - Test AUC (5-FU, CRC): {round(AUCTfluorouracil_crc,2)}")
AUCTfluorouracil_crc_aucpr = average_precision_score(Ytsfluorouracil_crc.astype(int), PredTfluorouracil_crc.detach().numpy())
#print(f"PDX - Test AUCPR (5-FU, CRC): {round(AUCTfluorouracil_crc_aucpr,2)}\n")

AUCT_finetuned = dict()
AUCTaucpr_finetuned = dict()
AUCTfluorouracil_finetuned = dict()
AUCTfluorouracil_crc_finetuned = dict()
AUCTfluorouracil_aucpr_finetuned = dict()
AUCTfluorouracil_crc_aucpr_finetuned = dict()
for iter in range(max_iter):
   iter += 1  

   AutoencoderE_finetuned = torch.load(models_dir + f"Finetuned_Models/Exprs_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_{iter}.pt")
   AutoencoderC_finetuned = torch.load(models_dir + f"Finetuned_Models/CNA_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_{iter}.pt")
   Clas_finetuned = torch.load(models_dir + f"Finetuned_Models/Class_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_{iter}.pt")
   
   AutoencoderE_finetuned.eval()
   AutoencoderC_finetuned.eval()
   Clas_finetuned.eval()
      
   ZEX = AutoencoderE_finetuned(torch.FloatTensor(X_trainE))
   ZCX = AutoencoderC_finetuned(torch.FloatTensor(X_trainC))
   ZTX = torch.cat((ZEX, ZCX), 1)
   ZTX = F.normalize(ZTX, p=2, dim=0)
   PredX = Clas_finetuned(ZTX)
   AUCT_finetuned[iter] = roc_auc_score(Y, PredX.detach().numpy())
   #print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT,2)}")
   AUCTaucpr_finetuned[iter] = average_precision_score(Y, PredX.detach().numpy())
   #print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr,2)}")
   
   ZETfluorouracil = AutoencoderE_finetuned(TX_testEfluorouracil)
   ZCTfluorouracil = AutoencoderC_finetuned(TX_testCfluorouracil)
   ZTTfluorouracil = torch.cat((ZETfluorouracil, ZCTfluorouracil), 1)
   ZTTfluorouracil = F.normalize(ZTTfluorouracil, p=2, dim=0)
   PredTfluorouracil = Clas_finetuned(ZTTfluorouracil)
   AUCTfluorouracil_finetuned[iter] = roc_auc_score(Ytsfluorouracil.astype(int), PredTfluorouracil.detach().numpy())
   #print(f"PDX - Test AUC (5-FU, Finetuned = {iter + 1}): {round(AUCTfluorouracil,2)}")
   AUCTfluorouracil_aucpr_finetuned[iter] = average_precision_score(Ytsfluorouracil.astype(int), PredTfluorouracil.detach().numpy())
   #print(f"PDX - Test AUCPR (5-FU, Finetuned = {iter + 1}): {round(AUCTfluorouracil_aucpr,2)}\n")

   ZETfluorouracil_crc = AutoencoderE_finetuned(TX_testEfluorouracil_crc)
   ZCTfluorouracil_crc = AutoencoderC_finetuned(TX_testCfluorouracil_crc)
   ZTTfluorouracil_crc = torch.cat((ZETfluorouracil_crc, ZCTfluorouracil_crc), 1)
   ZTTfluorouracil_crc = F.normalize(ZTTfluorouracil_crc, p=2, dim=0)
   PredTfluorouracil_crc = Clas_finetuned(ZTTfluorouracil_crc)
   AUCTfluorouracil_crc_finetuned[iter] = roc_auc_score(Ytsfluorouracil_crc.astype(int), PredTfluorouracil_crc.detach().numpy())
   AUCTfluorouracil_crc_aucpr_finetuned[iter] = average_precision_score(Ytsfluorouracil_crc.astype(int), PredTfluorouracil_crc.detach().numpy())

max_key = max(AUCTfluorouracil_aucpr_finetuned, key=AUCTfluorouracil_aucpr_finetuned.get)
max_key_crc = max(AUCTfluorouracil_crc_aucpr_finetuned, key=AUCTfluorouracil_crc_aucpr_finetuned.get)

print(f"GDSC - Training AUC (DNA_REPi): {round(AUCT,2)}")
print(f"GDSC - Training AUCPR (DNA_REPi): {round(AUCTaucpr,2)}")
print(f"PDX - Test AUC (5-FU): {round(AUCTfluorouracil,2)}")
print(f"PDX - Test AUCPR (5-FU): {round(AUCTfluorouracil_aucpr,2)}\n")
print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key],2)}")
print(f"PDX - Test AUC (5-FU, Finetuned): {round(AUCTfluorouracil_finetuned[max_key],2)}")
print(f"PDX - Test AUCPR (5-FU, Finetuned): {round(AUCTfluorouracil_aucpr_finetuned[max_key],2)}\n\n")

print("Models for maximum finetuning")
print(f"Exprs_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_{max_key}.pt")
print(f"CNA_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_{max_key}.pt")
print(f"Class_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_{max_key}.pt")

print("\nModels for maximum finetuning (fluorouracil, CRC)")
print(f"Exprs_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_{max_key_crc}.pt")
print(f"CNA_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_{max_key_crc}.pt")
print(f"Class_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_{max_key_crc}.pt")

GDSC - Training AUC (DNA_REPi): 0.7
GDSC - Training AUCPR (DNA_REPi): 0.21
PDX - Test AUC (5-FU): 0.91
PDX - Test AUCPR (5-FU): 0.33

GDSC - Training AUC (DNA_REPi, Finetuned): 0.72
GDSC - Training AUCPR (DNA_REPi, Finetuned): 0.22
PDX - Test AUC (5-FU, Finetuned): 0.95
PDX - Test AUCPR (5-FU, Finetuned): 0.5


Models for maximum finetuning
Exprs_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_18.pt
CNA_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_18.pt
Class_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_18.pt

Models for maximum finetuning (fluorouracil, CRC)
Exprs_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_18.pt
CNA_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_18.pt
Class_Fluorouracil_GDSC_PDX_DNA_REPi_Expression_CNA_Finetuned_18.pt


In [47]:
# create a dataframe with the prediction results 

ec = {"Data": ["Expr + CNA", "Expr + CNA (Fine-tuned)"],
       "AUC (GDSC)": [round(AUCT, 2), round(AUCT_finetuned[max_key], 2)],
       "AUCPR (GDSC)": [round(AUCTaucpr,2), round(AUCTaucpr_finetuned[max_key],2)],
       "AUC (PDX)": [round(AUCTfluorouracil,2), round(AUCTfluorouracil_finetuned[max_key],2)],
       "AUCPR (PDX)":  [round(AUCTfluorouracil_aucpr,2), round(AUCTfluorouracil_aucpr_finetuned[max_key],2)],
       "AUC (PDX-CRC)":  [round(AUCTfluorouracil_crc,2), round(AUCTfluorouracil_crc_finetuned[max_key_crc],2)],
       "AUCPR (PDX-CRC)": [round(AUCTfluorouracil_crc_aucpr,2), round(AUCTfluorouracil_crc_aucpr_finetuned[max_key_crc],2)],
       "Sample Size (GDSC)": [GDSCEv2.shape[0], GDSCEv2.shape[0]],
       "Feature Size (GDSC)": [GDSCEv2.shape[1], GDSCEv2.shape[1]] 
}
ec_dataframe = pd.DataFrame.from_dict(ec, orient='index').transpose() 

ec_dataframe.to_csv(save_results_to + "GDSC_PDX_Expression_CNA_DNA_REPi_5-Fluorouracil.tsv",
                    sep = "\t",
                    index = False)

ec_dataframe

,Data,AUC (GDSC),AUCPR (GDSC),AUC (PDX),AUCPR (PDX),AUC (PDX-CRC),AUCPR (PDX-CRC),Sample Size (GDSC),Feature Size (GDSC)
0,Expr + CNA,0.7,0.21,0.91,0.33,0.91,0.33,9435,849
1,Expr + CNA (Fine-tuned),0.72,0.22,0.95,0.5,0.95,0.5,9435,849


In [48]:
# show expression layer parameters

AutoencoderE

AEE(
  (EnE): Sequential(
    (0): Linear(in_features=849, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.6, inplace=False)
  )
)

In [49]:
# show CNA layer parameters

AutoencoderC

AEC(
  (EnC): Sequential(
    (0): Linear(in_features=849, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.6, inplace=False)
  )
)

In [50]:
# show classification layer parameters

Clas

Classifier(
  (FC): Sequential(
    (0): Linear(in_features=640, out_features=1, bias=True)
    (1): Dropout(p=0.7, inplace=False)
    (2): Sigmoid()
  )
)

In [51]:
# change directory

os.chdir(save_results_to)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/PDX_DNA_REPi/Expression_CNA/Predictions


In [52]:
# save predictions

file = open('GDSC_PDX_DEGs - PDX_DNA_REPi_5-Fluorouracil Predictions (Expression & CNA).txt', 'w')
file.write(f"GDSC Training (EC) AUC (PDX_DNA_REPi): {round(AUCT,2)}\n")
file.write(f"GDSC Training (EC) AUCPR (PDX_DNA_REPi): {round(AUCTaucpr,2)}\n\n")

file.write(f"PDX Test (EC) AUC (5-Fluorouracil): {round(AUCTfluorouracil,2)}\n")
file.write(f"PDX Test (EC) AUCPR (5-Fluorouracil): {round(AUCTfluorouracil_aucpr,2)}\n\n")

file.close()